### Este notebook possui as seguintes finalidades

1 - Consultar os arquivos da pasta Staging no Google Drive.
2 - Pegar todos os arquivos e transformar em DF.
3 - Retornar os cabeçalhos.
4 - Gerar CSV final
5 - Armazenar os arquivos .dat na pasta correta 1m

In [2]:
# Trabalhar com DF
import pandas as pd
# Acessar sistema operacional
import os
# Irá retornar uma lista de arquivos
import glob
# Manipulação de arquivos e diretórios
import shutil
# Identificar o separador do arquivo
import csv

###### Todos os caminhos das pastas

In [3]:
# Onde esses arquivos serão armazenados
pasta_Staging = r'I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\Staging'
# Arquivos de 1 minuto
pasta_Gdrive_1m = r'I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\T1_Meteo_1m_2023\Ascii'
# Caminho csv 1m
arquivo_csv = r'I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\T1_Meteo_1m_2023.csv'

###### Arquivos de 1 Minuto

In [4]:
# Obter a lista de arquivos .dat na pasta 1 minuto
arquivos_staging = glob.glob(f'{pasta_Staging}\\*.dat')

# Se não houve nenhum arquivo
if not arquivos_staging:
    print('Erro: A pasta está vazia.')
    exit() # Terminar o programa.

Erro: A pasta está vazia.


In [1]:
# Há algumas linhas com erro no CSV, irei pula-las e relatar 
linhas_problematicas = []

try:
    df = pd.read_csv(arquivo_csv,sep=",",error_bad_lines=False)
except pd.errors.ParserErros as e:
    linhas_com_erros = [int(line.split()[0]) for line in str(e).split('\n')[1:-1]]
    
    # Adicione os índices das linhas problemáticas à lista linhas_problematicas
    linhas_problematicas.extend(linhas_com_erros)
    
print("Índices das linhas problemáticas:", linhas_problematicas)

NameError: name 'pd' is not defined

##### Teste para verificar se todas as colunas estão presentes nos arquivos

In [51]:
# Criação da função que aceita os dois parâmetros
def test_all_col_exists(df_1,cols):
# Compreensão de lista que percorre cada elemento da lista
    missing_cols = [col for col in cols if col not in df_1.columns]
    
    # O print irá trazer as colunas que não estão presentes no DF
    if missing_cols:
        print(f'A seguintes colunas não foram encontradas no {df_1}: {missing_cols}')
    else:
        print('Todas as colunas foram encontradas no DF')
        
# Colunas para verificação 
colunas_verificadas = [ 'TIMESTAMP', 'RECORD', 'Batt_volt_Min', 'PTemp', 'WindSpeed_1', 'WindDir_1', 'WindSpeed_2', 'WindDir_2',
    'WindSpeed_3', 'WindDir_3', 'WindSpeed_4', 'WindDir_4', 'temp_1_Avg', 'temp_2_Avg', 'temp_3_Avg', 'temp_4_Avg',
    'humid_1_Avg', 'humid_2_Avg', 'humid_3_Avg', 'humid_4_Avg', 'par1_in_Avg', 'par2_in_Avg', 'par3_in_Avg',
    'par4_in_Avg', 'chuva_Tot', 'SBTempC_1_Avg', 'TargTempC_1_Avg', 'SBTempC_2_Avg', 'TargTempC_2_Avg',
    'SBTempC_3_Avg', 'TargTempC_3_Avg', 'SBTempC_4_Avg', 'TargTempC_4_Avg', 'pressao_Avg', 'rad_total_Avg',
    'rad_diff_Avg']

In [52]:
# Variável para armazenar parte do nome do arquivo
meteo_1m = "CR6_T1_meteo_"
# Lista dos arquivos verificados para ser utilizada posteriormente
arquivos_verificados = []


for arquivo in arquivos_staging:
    # Retornando apenas o nome do arquivo sem o caminho da pasta
    nome_arquivo = os.path.basename(arquivo)
    
    if meteo_1m in nome_arquivo:
        
        df_1 = pd.read_csv(arquivo, sep=',',skiprows = [0,2,3]) #header=1)
        # Chama a função para verificar a presença das colunas
        test_all_col_exists(df_1, colunas_verificadas)
        arquivos_verificados.append(nome_arquivo)
        
print(f'{len(arquivos_verificados)} arquivos foram verificados')

Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas foram encontradas no DF
Todas as colunas

In [53]:
# Consultar a quantidade de linhas do arquivo csv
print(df.shape[0])

340843


##### Função teste

In [54]:
# Função para detectar o separados dos arquivos
def detectar_separador(arquivo):
    with open(arquivo, 'r') as file:
        conteudo = file.read(1024)  # Leitura dos primeiros 1024 bytes para detectar o separador
        if ';' in conteudo:
            return ';'
        elif ',' in conteudo:
            return ','
        else:
            return '\t'  # Use '\t' como separador padrão se nenhum for detectado


In [55]:
"""separdor_virgula = []
separador_ponto = []
separador_tabula = []

fro"""

'separdor_virgula = []\nseparador_ponto = []\nseparador_tabula = []\n\nfro'

##### Gerar o arquivo csv sem duplicatas

In [56]:
soma_linhas = 0
nome_csv = R'I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\Staging\Teste.csv'

# Crie um DataFrame vazio para iniciar
df_final = pd.DataFrame()

for arquivo in arquivos_staging:
    #separador = detectar_separador(arquivos_verificados)
    nome_arquivo = os.path.basename(arquivo)
    if nome_arquivo in arquivos_verificados:
    
         # Ler o arquivo usando o separador detectado
        df_1 = pd.read_csv(arquivo,sep=detectar_separador(arquivo), skiprows=[0, 2, 3])
    
    # Atualize o contador de linhas
        soma_linhas += df_1.shape[0]
    # Concatene o DataFrame lido ao df_final existente para cada iteração
        df_final = pd.concat([df_final,df_1],ignore_index=True)

# Gerando o df_final sem duplicatas
df_sem_duplicatas = df_final.drop_duplicates()

# Ordenando o DF pela coluna Record
df_sem_duplicatas = df_sem_duplicatas.sort_values(by="RECORD")
# Resetar o indice
df_sem_duplicatas = df_sem_duplicatas.reset_index(drop=True)

df_sem_duplicatas.head()
# Use o método to_csv para salvar o DataFrame em um arquivo CSV
df_sem_duplicatas.to_csv(nome_csv,sep=",", index=False)
#print(f'Arquivo {nome_arquivo} gerado com sucesso.')

##### Realizar a leitura do arquivo CSV gerado, passo não necessário 

In [44]:
"""nome_arquivo = "Teste.csv"

with open(nome_arquivo, 'w', newline='') as csvfile:
    # Especifique o delimitador de células (pode ser tabulação, ponto e vírgula, etc.)
    delimitador = '\t'  # Neste exemplo, usamos tabulação como delimitador
    
    # Crie um escritor CSV usando o delimitador especificado
    csvwriter = csv.writer(csvfile, delimiter=delimitador)
    
    # Escreva os dados do DataFrame no arquivo CSV
    for _, linha in df_sem_duplicatas.iterrows():
        # Converte cada valor da linha em uma lista e escreve no arquivo
        csvwriter.writerow(linha.tolist())
"""

In [57]:
print(df_sem_duplicatas.shape[0])
print(soma_linhas)

51837
51837


##### Isso irá automatizar meu processo de incrementar o arquivo gerado, mas antes preciso gerar o arquivo csv final separado por virgulas

In [58]:
"""
# Concatenando ao longo das linhas Axis=0
df_concat = pd.concat([df,df_sem_duplicatas],axis=0)
df_to_csv = df_concat.sort_values(by="RECORD") 
df_to_csv = df_to_csv.reset_index(drop=True)
df_to_csv.to_csv(nome_csv, index=False)
print(f'Arquivo {nome_csv} gerado com sucesso.')
"""

'\n# Concatenando ao longo das linhas Axis=0\ndf_concat = pd.concat([df,df_sem_duplicatas],axis=0)\ndf_to_csv = df_concat.sort_values(by="RECORD") \ndf_to_csv = df_to_csv.reset_index(drop=True)\ndf_to_csv.to_csv(nome_csv, index=False)\nprint(f\'Arquivo {nome_csv} gerado com sucesso.\')\n'

In [59]:
print(df_to_csv.shape[0])

392680


##### Enviar os arquivos verificados para a pasta T1_1m

In [62]:
# Lista que armazena a quantidade de itens enviados
qntd_enviada = []
qntd_excluida = []

for arquivo in arquivos_staging:
    nome_arquivo = os.path.basename(arquivo)
    if nome_arquivo in arquivos_verificados:
        # Buscar o caminho desse arquivo na staging
        caminho_arquivo_staging = f'{pasta_Staging}\\{nome_arquivo}'
        # Buscar a pasta Staging no GDrive
        caminho_arquivo_gdrive = f'{pasta_Gdrive_1m}'
        # Copiar o arquivo de uma pasta para outra
        shutil.copy(caminho_arquivo_staging, caminho_arquivo_gdrive)
        # Para cada arquivo 1 mensagem
        print(f'Arquivo {nome_arquivo} copiado para Gdrive')
        # Adicionar +1 para a lista
        qntd_enviada.append(+1)
        # Código abaixo é para deletar os arquivos que foram transferidos para a outra pasta
        if os.path.exists(caminho_arquivo_staging):
            #os.remove é usado para excluir só o arquivo, não as pasta
            os.remove(caminho_arquivo_staging)
            print(f'Arquivo {caminho_arquivo_staging} excluído com sucesso')
            qntd_excluida.append(+1)
        else:
            print(f'O arquivo {caminho_arquivo_staging} não existe')
print(qntd_enviada)

Arquivo AmazonFACE_CR6_T1_meteo_2023_08_31_0000.dat copiado para Gdrive
Arquivo I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\Staging\AmazonFACE_CR6_T1_meteo_2023_08_31_0000.dat excluído com sucesso
Arquivo AmazonFACE_CR6_T1_meteo_2023_09_01_0000.dat copiado para Gdrive
Arquivo I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\Staging\AmazonFACE_CR6_T1_meteo_2023_09_01_0000.dat excluído com sucesso
Arquivo AmazonFACE_CR6_T1_meteo_2023_09_02_0000.dat copiado para Gdrive
Arquivo I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\Staging\AmazonFACE_CR6_T1_meteo_2023_09_02_0000.dat excluído com sucesso
Arquivo AmazonFACE_CR6_T1_meteo_2023_09_03_0000.dat copiado para Gdrive
Arquivo I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 2023\Staging\AmazonFACE_CR6_T1_meteo_2023_09_03_0000.dat excluído com sucesso
Arquivo AmazonFACE_CR6_T1_meteo_2023_09_04_0000.dat copiado para Gdrive
Arquivo I:\Meu Drive\AmazonFace\Dados das Torres do Face\T1\07 - 202

In [64]:
# Chama a função para verificar a presença das colunas
#test_all_col_exists(df, colunas_verificadas)
print(len(qntd_enviada))
print(len(qntd_excluida))

33


##### Próximo passo, fazer a leitura dos arquivos de 1m novamente e gerar o arquivo CSV separado por virgulas

In [2]:
print(df.shape[0])

NameError: name 'df' is not defined